<a href="https://colab.research.google.com/github/arunt-sjsu/deep_learning/blob/main/Assignment4/Tensorflow_without_Auto_primitives.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**The Goal of this notebook is to demonstrate tensorflow based linear regression to predict 'a' from the dummy data. This notebook does not use auto primitives**

In [5]:
%matplotlib inline
import numpy as np
import pandas as pd
import tensorflow as tf
print(tf.__version__)

2.4.1


In [6]:
df = pd.DataFrame(np.random.randn(10000, 3)*10, columns=list('xyz'))
weights = pd.DataFrame(np.random.randn(10000,1), columns=['weights_a'])
weights['weights_b'] = np.random.randn(10000,1)
weights['weights_c'] = np.random.randn(10000,1)

In [7]:
df["a"] = 2 * weights['weights_a'] * np.square(df['x']) + 8 * weights['weights_b'] * np.power(df['y'],3) - 3 * df['z'] 
df["b"] = 3 * weights['weights_a'] * np.square(df['x']) - 5 * weights['weights_b'] * np.square(df['y']) - 2 * weights['weights_c'] * np.power(df['z'],3) 

In [8]:
bin_labels = ['Low', 'Mid', 'High', 'Very High']
df['Level_A'] = pd.qcut(df['a'], q=4, labels=bin_labels)
df['Level_B'] = pd.qcut(df['b'], q=4, labels=bin_labels)


In [11]:
# Let's use gradient descent to learn the weights and bias that minimizes the loss function.
# For this, we need the gradient of the loss function and the gradients of the linear function.

class MSE:
  def __call__(self, y_pred, y_true):
    self.y_pred = y_pred
    self.y_true = y_true
    return ((y_pred - y_true) ** 2).mean()

  def backward(self):
    n = self.y_true.shape[0]
    self.gradient = 2. * (self.y_pred - self.y_true) / n
    # print('MSE backward', self.y_pred.shape, self.y_true.shape, self.gradient.shape)
    return self.gradient


class Linear:
  def __init__(self, input_dim: int, num_hidden: int = 1):
    self.weights = np.random.randn(input_dim, num_hidden) * np.sqrt(2. / input_dim)
    self.bias = np.zeros(num_hidden)
  
  def __call__(self, x):
    self.x = x
    output = x @ self.weights + self.bias
    return output

  def backward(self, gradient):
    self.weights_gradient = self.x.T @ gradient
    self.bias_gradient = gradient.sum(axis=0)
    self.x_gradient = gradient @ self.weights.T
    return self.x_gradient

  def update(self, lr):
    self.weights = self.weights - lr * self.weights_gradient
    self.bias = self.bias - lr * self.bias_gradient

In [21]:
d = 3
x = df[["x","y","z"]]
bias_true = np.array([0.5])
print(x.shape, bias_true.shape)

(10000, 3) (1,)


In [22]:
loss = MSE()
linear = Linear(3)
y_pred = linear(x)
print(loss(y_pred, y_true))

0    763.169197
dtype: float64


In [23]:
from typing import Callable

def fit(x: np.ndarray, y: np.ndarray, model: Callable, loss: Callable, lr: float, num_epochs: int):
  for epoch in range(num_epochs):
    y_pred = model(x)
    loss_value = loss(y_pred, y)
    print(f'Epoch {epoch}, loss {loss_value}')
    gradient_from_loss = loss.backward()
    model.backward(gradient_from_loss)
    model.update(lr)

fit(x, y_true, model=linear, loss=loss, lr=0.1, num_epochs=20)

Epoch 0, loss 0    763.169197
dtype: float64
Epoch 1, loss 0    280043.140618
dtype: float64
Epoch 2, loss 0    1.028788e+08
dtype: float64
Epoch 3, loss 0    3.782597e+10
dtype: float64
Epoch 4, loss 0    1.391955e+13
dtype: float64
Epoch 5, loss 0    5.126714e+15
dtype: float64
Epoch 6, loss 0    1.889904e+18
dtype: float64
Epoch 7, loss 0    6.973241e+20
dtype: float64
Epoch 8, loss 0    2.575316e+23
dtype: float64
Epoch 9, loss 0    9.519930e+25
dtype: float64
Epoch 10, loss 0    3.522494e+28
dtype: float64
Epoch 11, loss 0    1.304624e+31
dtype: float64
Epoch 12, loss 0    4.836642e+33
dtype: float64
Epoch 13, loss 0    1.794857e+36
dtype: float64
Epoch 14, loss 0    6.667246e+38
dtype: float64
Epoch 15, loss 0    2.479115e+41
dtype: float64
Epoch 16, loss 0    9.227460e+43
dtype: float64
Epoch 17, loss 0    3.437988e+46
dtype: float64
Epoch 18, loss 0    1.282223e+49
dtype: float64
Epoch 19, loss 0    4.786959e+51
dtype: float64
